<a href="https://colab.research.google.com/github/AloneMusk/DeepLearning_with_PyTorch/blob/main/Simple_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [21]:
#Create a model class that inherits the nn.module

class Model(nn.Module):
  #Input(4 features of the flowers) -->
  #Hidden layer 1 (n neurons) -->
  #Hidden layer 2 (n neurons)-->
  #O/p layer 3 flowers that need to be classified
  def __init__(self, in_features=4, h1=8, h2=9, output_features=3):
    super().__init__() #instantiate the nn.Module
    self.fc1 = nn.Linear(in_features, h1) #fully connected layer 1 (meaning every node is connected from inputs to every node in first hidden layer)
    self.fc2 = nn.Linear(h1, h2) #fully connected layer 2
    self.out = nn.Linear(h2, output_features)

  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.out(x)

    return x



In [22]:
#Pick a manual seed for randomization
torch.manual_seed(32) #Seed value can affect NN performance

#Create an instance for our model
model = Model()

In [23]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [24]:
url = 'https://gist.githubusercontent.com/curran/a08a1080b88344b0c8a7/raw/0e7a9b0a5d22642a06d3d5b9bcbad9890c8ee534/iris.csv'
df = pd.read_csv(url)

In [25]:
#Change last column from strings to integers
df['species'] = df['species'].replace('setosa', 0)
df['species'] = df['species'].replace('versicolor', 1)
df['species'] = df['species'].replace('virginica', 2)

In [26]:
df

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [27]:
#Train, Test and split
X = df.drop('species', axis = 1)
y = df['species']

In [ ]:
#Convert these to numpy arrays
X = X.values
y = y.values
X,y

In [29]:
from sklearn.model_selection import train_test_split

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 32)

In [31]:
#Convert numpy arrays into tensors
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

In [32]:
#Set the criteria of our model to measure the error
criterion = nn.CrossEntropyLoss()

#Choose an optimizer, learning rate
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [33]:
print(model.parameters)

<bound method Module.parameters of Model(
  (fc1): Linear(in_features=4, out_features=8, bias=True)
  (fc2): Linear(in_features=8, out_features=9, bias=True)
  (out): Linear(in_features=9, out_features=3, bias=True)
)>


In [34]:
#Train the model
#1. epochs -> one run through all the training data in our data
epochs = 200
losses = []
for i in range(epochs):
  #GO forward and predict
  y_pred = model.forward(X_train) #Get predicted results

  #measure the loss
  loss = criterion(y_pred, y_train)

  #keep track of losses
  losses.append(loss.detach().numpy())

  #print every 10 epochs
  if i%10 == 0:
    print(f'Epoch: {i} and loss: {loss}')

  #Do some backprop
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

Epoch: 0 and loss: 1.169895887374878
Epoch: 10 and loss: 0.9578844308853149
Epoch: 20 and loss: 0.7988812923431396
Epoch: 30 and loss: 0.634953498840332
Epoch: 40 and loss: 0.44043001532554626
Epoch: 50 and loss: 0.29171693325042725
Epoch: 60 and loss: 0.17829042673110962
Epoch: 70 and loss: 0.11587585508823395
Epoch: 80 and loss: 0.08928148448467255
Epoch: 90 and loss: 0.07743765413761139
Epoch: 100 and loss: 0.0712408497929573
Epoch: 110 and loss: 0.06744467467069626
Epoch: 120 and loss: 0.06482169777154922
Epoch: 130 and loss: 0.06285707652568817
Epoch: 140 and loss: 0.061298247426748276
Epoch: 150 and loss: 0.06000852957367897
Epoch: 160 and loss: 0.05890548229217529
Epoch: 170 and loss: 0.05793692544102669
Epoch: 180 and loss: 0.057068899273872375
Epoch: 190 and loss: 0.05627993121743202


In [35]:
#Evaluate model on Test Data Set
with torch.no_grad(): #Basically turns off backprop
  y_eval = model.forward(X_test)
  loss = criterion(y_eval, y_test) #measure loss

In [36]:
loss

tensor(0.0298)

In [37]:
correct = 0

with torch.no_grad():
  for i, data in enumerate(X_test):
    y_val = model.forward(data)

    print(f'{i+1}.) {str(y_val)} \t {y_test[i]}') #Tells us what type of flower our network thinks it is

    #Correct or not
    if y_val.argmax().item() == y_test[i]:
      correct+=1

print(f'We got {correct} correct')

1.) tensor([-4.5474,  5.6474,  0.6258]) 	 1
2.) tensor([  9.7839,   2.0130, -14.4508]) 	 0
3.) tensor([  8.7851,   2.2439, -13.3628]) 	 0
4.) tensor([-4.7777,  5.9443,  0.5937]) 	 1
5.) tensor([-9.7693,  3.8727,  7.0655]) 	 2
6.) tensor([-10.4429,   5.0144,   6.8463]) 	 2
7.) tensor([  8.1658,   2.5038, -12.7315]) 	 0
8.) tensor([  9.2193,   2.0836, -13.7833]) 	 0
9.) tensor([-4.0243,  5.9801, -0.2057]) 	 1
10.) tensor([  9.5090,   2.0918, -14.1647]) 	 0
11.) tensor([-4.9569,  6.4096,  0.5537]) 	 1
12.) tensor([-11.6292,   2.6205,   9.6212]) 	 2
13.) tensor([-2.2209,  5.4773, -1.5542]) 	 1
14.) tensor([-1.4594,  6.1748, -2.9049]) 	 1
15.) tensor([-9.9357,  3.6303,  7.3054]) 	 2
16.) tensor([-11.2093,   2.2251,   9.4723]) 	 2
17.) tensor([-5.2317,  5.1326,  1.6100]) 	 1
18.) tensor([-8.7368,  4.0250,  5.8872]) 	 2
19.) tensor([-2.0934,  5.9443, -2.0632]) 	 1
20.) tensor([ 10.2769,   2.1301, -15.1903]) 	 0
21.) tensor([  8.9939,   2.3817, -13.7532]) 	 0
22.) tensor([-12.5156,   3.6030,  

##Feed new data and test out the model

In [38]:
#[sepal_length,	sepal_width,	petal_length,	petal_width]
new_iris = torch.tensor([4.7, 4.2, 1.3, 0.2])

In [39]:
with torch.no_grad():
  print(model(new_iris)) #this one predicts the flower to be setosa given the different features of the flower that we randomly entered in new_iris

tensor([ 10.2631,   2.1269, -15.1697])


In [40]:
newer_iris = torch.tensor([6.0, 6.0, 6.0, 6.0])
with torch.no_grad():
  print(model(newer_iris))

tensor([-18.2133,  -0.5969,  17.9550])


##Save and load the NN model

In [41]:
#Save the model
torch.save(model.state_dict(), 'my_iris_model.pt') #state_dict stores all the weights and biases of our model in a dict so that we can use it later

In [42]:
#Load the model
new_model = Model()
new_model.load_state_dict(torch.load('my_iris_model.pt'))

<All keys matched successfully>

In [43]:
#Verify if the values match
new_model.eval()

Model(
  (fc1): Linear(in_features=4, out_features=8, bias=True)
  (fc2): Linear(in_features=8, out_features=9, bias=True)
  (out): Linear(in_features=9, out_features=3, bias=True)
)

In [44]:
#You can also verify by checking the predictions with diff data points
with torch.no_grad():
  print(new_model(newer_iris))

tensor([-18.2133,  -0.5969,  17.9550])
